# Human and AI Messages

In this notebook you'll learn about two of the core chat message types, human and AI messages, and how to use them explictly in application code.

---

## Objectives

By the time you complete this notebook you will:

- Make explicit the role-based messaging system utilized by chat variant LLMs.
- Learn how to use `ChatPromptTemplate` to create human and AI messages in prompt templates.

---

## Imports

In [1]:
!pip install groq langchain-groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.9/134.9 kB 3.0 MB/s eta 0:00:00


In [2]:
import os
import getpass

os.environ["GROQ_API_KEY"] = getpass.getpass("GROQ API Key:\n")

GROQ API Key:
··········


In [4]:
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import HumanMessage, AIMessage

---

## Create a Model Instance

In [5]:
llm = ChatGroq(model_name="llama-3.3-70b-versatile", temperature=0)

---

## Chat Messages

As we've mentioned earlier in the workshop, there are important differences working with LLMs that are of the "chat" or "instruction" variety, versus LLMs that are not. Primarily, we have pointed out that chat models are intended to be used in a conversational manner whereas non-chat models primarily aim to generate whatever ought to come next for a given piece of text.

As we have been working with LangChain throughout the workshop, we have been constructing prompts primarily via `ChatPromptTemplate`. As the "Chat" in the name implies, prompts constructed with `ChatPromptTemplate` are well-suited to creating prompts that work with the expectations of chat models, namely, role-based conversational interactions.

Later in this notebook we are going to learn how to leverage our understanding of the various roles involved in chat model interactions, but to begin, let's revisit some familiar tasks, but take the time to notice how our prompts and our chat model's responses are in fact being structured in a way to indicate that they are in fact a part of a role-based dialogue.

---

## Human Messages

To begin, we'll create an incredibly simple chat prompt template.

In [6]:
prompt_template = ChatPromptTemplate.from_template("{prompt}")

Next we'll instantiate an actual prompt by invoking the prompt template and then print the prompt in its entirety so we can discuss it further in the context of our current exploration.

In [7]:
prompt = prompt_template.invoke({"prompt": "hello"})

In [8]:
prompt

ChatPromptValue(messages=[HumanMessage(content='hello', additional_kwargs={}, response_metadata={})])

First, we see that this prompt is a `ChatPromptValue`: this is a prompt intended to be used with chat model.

Next, we notice that the message is something called a `HumanMessage`. Messages in a chat dialogue are always associated with a role, and this message is understood to be generated by someone with the role of "human".

Invoking the `to_messages` method on the prompt makes this even more clear.

In [9]:
prompt.to_messages()

[HumanMessage(content='hello', additional_kwargs={}, response_metadata={})]

---

## AI Messages

Let's create a very basic chain so we can send our prompt to our chat model and then take a closer look at the message it sends back to us. Worth noting is that we are not including a parser at the end of our chain because we are interested to explore the entirety of the response from the model.

In [10]:
chain = prompt_template | llm

In [11]:
response = chain.invoke({"prompt": "hello"})

In [12]:
response

AIMessage(content='Hello. How can I assist you today?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 36, 'total_tokens': 46, 'completion_time': 0.017223443, 'prompt_time': 0.010836767, 'queue_time': 0.275482618, 'total_time': 0.02806021}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_2ddfbb0da0', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--33a94596-f5a0-4af0-8b32-bdc5153b3d0e-0', usage_metadata={'input_tokens': 36, 'output_tokens': 10, 'total_tokens': 46})

Right away we see that the response from the model is an `AIMessage` which we (and the model) can take to understand means that the message was generated by someone with the role of "AI".

---

## Explicit Role Use in Prompts

So under the hood LangChain, via `ChatPromptTemplate` has been taking care to manage the roles of both our "human" prompts and the "AI" chat model responses. However, LangChain also provides us with easy to use mechanisms for explicit role management.

One of the most simple ways is by using `ChatPromptTemplate.from_messages`, which takes a list of messages, where each message is a 2-tuple with its first value indicating the role associated with the message, and the second value being the content of the message itself.

Here we use `ChatPromptTemplate.from_messages` to recreate the exact same prompt we created above using `from_template`. In this case we are explicitly stating that the prompt will be associated with the "human" role.

In [13]:
prompt_template = ChatPromptTemplate.from_messages([
    ("human", "{prompt}")
])

Looking at the prompt itself we see that just like what we did above, the the prompt is a `ChatPromptValue` that includes a message of type `HumanMessage`.

In [14]:
prompt = prompt_template.invoke({"prompt": "hello"})

In [15]:
prompt

ChatPromptValue(messages=[HumanMessage(content='hello', additional_kwargs={}, response_metadata={})])

In [16]:
prompt.to_messages()

[HumanMessage(content='hello', additional_kwargs={}, response_metadata={})]

We can work with this prompt template exactly like we've been doing all this time with prompts created with `from_template`.

In [17]:
chain = prompt_template | llm

In [18]:
response = chain.invoke({"prompt": "hello"})

In [19]:
response

AIMessage(content='Hello. How can I assist you today?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 36, 'total_tokens': 46, 'completion_time': 0.010035836, 'prompt_time': 0.010457354, 'queue_time': 0.199225146, 'total_time': 0.02049319}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_2ddfbb0da0', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--87c56c52-bb8e-42df-9328-924a7ad471cb-0', usage_metadata={'input_tokens': 36, 'output_tokens': 10, 'total_tokens': 46})

---

## Using ChatPromptMessages Directly

As an aside, in more recent versions of LangChain we can use `ChatPromptMessages` directly, which is equivalent to using `ChatPromptTemplate.from_messages`. Thus the following two cells are identical.

In [20]:
ChatPromptTemplate.from_messages([
    ("human", "{prompt}")
])

ChatPromptTemplate(input_variables=['prompt'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['prompt'], input_types={}, partial_variables={}, template='{prompt}'), additional_kwargs={})])

In [21]:
ChatPromptTemplate([
    ("human", "{prompt}")
])

ChatPromptTemplate(input_variables=['prompt'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['prompt'], input_types={}, partial_variables={}, template='{prompt}'), additional_kwargs={})])

We will primarily use `from_messages` in this workshop primarily because we expect you to see it much more frequently in documentation and literature, but feel free to use whichever of the two variations you like.

---

## Exercise: Create an Explicit Human Message

As a very simple exercise, just to get you actively using human messages explicitly, refactor the following chain to use `ChatPromptTemplate.from_messages`.

In [22]:
template = ChatPromptTemplate.from_template("Give the concise etomology of the following English word: {word}")

In [23]:
parser = StrOutputParser()

In [24]:
chain = template | llm | parser

In [25]:
print(chain.invoke({"word": "learning"}))

The word "learning" comes from Old English "leornian" (to learn) and "learnung" (a learning), which are derived from Proto-Germanic "*liznojan" and further from Proto-Indo-European "*leis-" (to track, follow).


### Your Work Here

### Solution

In [26]:
template = ChatPromptTemplate.from_messages([
    ("human", "Give the concise etomology of the following English word: {word}")
])

In [27]:
parser = StrOutputParser()

In [28]:
chain = template | llm | parser

In [29]:
print(chain.invoke({"word": "learning"}))

The word "learning" comes from Old English "leornian" (to learn) and "learnung" (a learning), which are derived from Proto-Germanic "*liznojan" and further from Proto-Indo-European "*leis-" (to track, follow).


---

## Explicit Use of the AI Role

In addition to providing "human" role message, we can also pass messages to the model associated with the AI role.

In [30]:
prompt_template = ChatPromptTemplate.from_messages([
    ("human", "Hello."),
    ("ai", "Hello, how are you?"),
    ("human", "{prompt}")
])

If we invoke this prompt we can see that unlike prompts we have sent to our chat models in the past, it contains 3 messages, two associated with the human role and one with the AI.

In [31]:
prompt = prompt_template.invoke({"prompt": "I'm well, thanks!"})

In [32]:
prompt.to_messages()

[HumanMessage(content='Hello.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Hello, how are you?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content="I'm well, thanks!", additional_kwargs={}, response_metadata={})]

This ability allows us to construct prompts that contain additional context for the model to use when generating its response.

From the model's perspective it is seeing what has already happened in the current chat conversation and this context about what has already happened can influence how the model responds in subsequent conversational turns.

There are 2 primary ways we tend to utilize this ability.

The first is when we want to implement chatbot functionality. After every human / AI interaction, we can add the interaction to our prompt. Thus, whenever we send a message to the chatbot, it is aware of the full context of the conversation up until that point, and is able to respond more appropriately. We will be looking at creating chatbot functionality in detail later in the workshop.

The second is by constructing our own fictitious human / AI interactions to serve as examples to the model about how it ought to respond to subsequent human messages. This technique of providing example human / AI interactions by way of our prompt is referred to as few-shot prompting, which we will look at in the next notebook.

---

## Using `HumanMessage` and `AIMessage`

It's often the case with LangChain that we have multiple ways to do the same thing, and this is true in the case of creating role-specific messages. Up until now we have been using the 2-tuple syntax to create a message explicitly associated with a role.

In [33]:
prompt_template = ChatPromptTemplate.from_messages([
    ("human", "Hello."),
    ("ai", "Hello, how are you?"),
    ("human", "{prompt}")
])

As an alternative to the 2-tuple syntax above, we can also use LangChain's `HumanMessage` and `AIMessage` classes.

In [34]:
from langchain_core.messages import HumanMessage, AIMessage

The following is identical to the 2-tuple implementation above.

In [35]:
prompt_template = ChatPromptTemplate.from_messages([
    HumanMessage(content="Hello"),
    AIMessage(content="Hello, how are you?"),
    HumanMessage(content="{prompt}")
])

It really is just a matter of choice which of these you might prefer to use in your applications, and you should feel free to use either in this workshop. Most importantly is to be able to recognize and understand each of them, as you're likely to see them both in documentation and examples.

## Summary

Now that you're familiar with human and AI messages, including how to author them in chat prompt templates, in the next notebook you'll learn a powerful and popular technique called few-shot prompting that will leverage your message authoring skills to provide chat models with examples capable of impacting their behavior.